In [ ]:
#! conda install -y gdal

In [ ]:
import os
import boto3
import logging
import shutil
from datetime import datetime

In [ ]:
s3_bucket_input = 'tiling-function-input-bucket'
s3_bucket_output = 'tiling-function-output-tiles'
window_size = '500 500'

#Temp directory
input_dir  = 'input_images/'
output_dir = 'output_tiles/'

In [ ]:
#Initializing the Logger and S3 libraries

logger = logging.getLogger()
#logger.setLevel(logging.INFO)
logger.setLevel(logging.DEBUG)

s3 = boto3.client('s3')

In [ ]:
#Creating the input and output folders

try:
    os.mkdir(input_dir)
    os.mkdir(output_dir)
except OSError as error:
    print(error)

In [ ]:
#Downloading the images from the S3 bucket into the local directory
logger.debug(f"Downloading the images from the S3 bucket has started.")

for obj in s3.list_objects(Bucket=s3_bucket_input)['Contents']:
    
    if (obj['Key'].split('.')[1] != 'jp2'):
        continue
    
    image_name = obj['Key']
    print(image_name)
    
    s3.download_file(s3_bucket_input, image_name, input_dir + image_name)
    
logger.debug(f"Images downloaded successfully into the local input folder.")

In [ ]:
! find ../../ -name gdal_retile.py

In [ ]:
gdal_script_dir = '/home/ec2-user/anaconda3/envs/python3/bin/gdal_retile.py'

In [ ]:
#Processing each image and saving the results into the output folder
images = os.listdir(input_dir)
print(images)

for image_name in images:
    
    if image_name == '.ipynb_checkpoints':
        continue
    
    target_dir = output_dir + image_name.split('.')[0] + '_tiles'
    try:
        #Creating a folder same as the name of the image
        os.mkdir(target_dir)
    except OSError as error:
        print(error)

    command = "python %s -ps %s -targetDir %s %s" % (gdal_script_dir, window_size, target_dir, input_dir + image_name)

    print(command)

    result = os.system(command)
    
    print(result)
    
    if (result == 1):
        logger.info("tiles have been generated successfully.")
    else:
        logger.info("Error occurred for generating the tiles.")

In [ ]:
#Upload the resutls to S3

folder_list = os.listdir(output_dir)
#print(folder_list)

for folder in folder_list:
    
    image_list = os.listdir(output_dir + folder)
    #print(image_list)
    
    for image_name in image_list:
        
        if image_name == '.ipynb_checkpoints':
            continue
        
        upload_file_path = output_dir + folder + '/' + image_name
        
        #print(upload_file_path)
        
        s3.upload_file(upload_file_path, 
                       s3_bucket_output, 
                       #datetime.today().strftime('%d-%m-%Y') + '/' + folder + '/' + image_name)
                       folder + '/' + image_name)


In [ ]:
#Removing the files from the input and output directories
shutil.rmtree(input_dir)
shutil.rmtree(output_dir)
logger.info('Removed the temp folders.')